In [1]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/train_preprocessed.csv')

In [3]:
val_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/val_preprocessed.csv')

In [4]:
train_X = train_df['preprocessed_text'].to_list()
train_Y = (train_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [5]:
val_X = val_df['preprocessed_text'].to_list()
val_Y = (val_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [6]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_X)

In [36]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=256,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [37]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,732,673
Trainable params: 2,732,673
No

In [39]:
history = model.fit(x = train_X, y = train_Y, epochs=10,
                    validation_data=(val_X, val_Y),
                    validation_steps=100)

Epoch 1/10
331/331 [==============================] - ETA: 0s - loss: 0.6321 - accuracy: 0.6710

331/331 [==============================] - 46s 118ms/step - loss: 0.6321 - accuracy: 0.6710 - val_loss: 0.6157 - val_accuracy: 0.6545
Epoch 2/10
331/331 [==============================] - 34s 103ms/step - loss: 0.5375 - accuracy: 0.6951
Epoch 3/10
331/331 [==============================] - 37s 111ms/step - loss: 0.3827 - accuracy: 0.8291
Epoch 4/10
331/331 [==============================] - 34s 104ms/step - loss: 0.2858 - accuracy: 0.8841
Epoch 5/10
331/331 [==============================] - 34s 101ms/step - loss: 0.2226 - accuracy: 0.9132
Epoch 6/10
331/331 [==============================] - 34s 103ms/step - loss: 0.1786 - accuracy: 0.9332
Epoch 7/10
331/331 [==============================] - 34s 103ms/step - loss: 0.1481 - accuracy: 0.9473
Epoch 8/10
331/331 [==============================] - 34s 102ms/step - loss: 0.1226 - accuracy: 0.9569
Epoch 9/10
331/331 [==============================] - 34s 103ms/step - loss: 0.1048 - accuracy: 0.9637
Epoch 10/10
331/331 [=====================

In [40]:
val_pred = model.predict(val_X)

83/83 [==============================] - 4s 27ms/step


In [41]:
val_loss, val_acc = model.evaluate(val_X, val_Y)

print('val Loss:', val_loss)
print('val Accuracy:', val_acc)

83/83 [==============================] - 2s 25ms/step - loss: 1.2757 - accuracy: 0.7217
val Loss: 1.2757498025894165
val Accuracy: 0.721676766872406


In [42]:
val_pred = np.where(val_pred > 0.5, 1, 0)

In [43]:
from sklearn.metrics import f1_score

f1_score(val_Y, val_pred, average='macro')

0.6868567595951038